# SQLAlchemy

In [1]:
# Import all necessary modules

import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker, relationship, backref, with_polymorphic

engine = sa.create_engine("sqlite:///:memory:")
Base = declarative_base()

In [2]:
# Declare All Models

class Employee(Base):
    __tablename__ = 'employee'
    id = sa.Column(sa.Integer, primary_key=True)
    name = sa.Column(sa.String(50))
    type = sa.Column(sa.String(50))

    __mapper_args__ = {
        'polymorphic_identity':'employee',
        #'with_polymorphic':"*", # REMOVE THIS AND CHECK OUTPUT
        'polymorphic_on':type
    }

class Engineer(Employee):
    __tablename__ = 'engineer'
    id = sa.Column(sa.Integer, sa.ForeignKey('employee.id'), primary_key=True)
    engineer_name = sa.Column(sa.String(30))

    __mapper_args__ = {
        'polymorphic_identity':'engineer',
    }
    

class InfraEngineer(Engineer):
    __tablename__ = 'infraengineer'
    id = sa.Column(sa.Integer, sa.ForeignKey('engineer.id'), primary_key=True)
    db_password = sa.Column(sa.String(30))

    __mapper_args__ = {
        'polymorphic_identity':'infraengineer',
    }

class Manager(Employee):
    __tablename__ = 'manager'
    id = sa.Column(sa.Integer, sa.ForeignKey('employee.id'), primary_key=True)
    manager_name = sa.Column(sa.String(30))

    __mapper_args__ = {
        'polymorphic_identity':'manager',
    }

    
# Create all metadata
Base.metadata.create_all(engine)

In [3]:
# Seed Data
session = scoped_session(sessionmaker(bind=engine))

emp1 = Employee(name="Serena")

eng1 = Engineer(name="Mariappan", engineer_name="Maari")
eng2 = Engineer(name="Jarvis", engineer_name="Jar")

inf1 = InfraEngineer(name="Yong Wen", engineer_name="Yong", db_password="password")
inf2 = InfraEngineer(name="Chris", engineer_name="Ch", db_password="12345")

man1 = Manager(name="Linus", manager_name="Li")
man2 = Manager(name="Feng", manager_name="Fe")


session.add(emp1)
session.add(eng1)
session.add(eng2)
session.add(inf1)
session.add(inf2)
session.add(man1)
session.add(man2)

session.commit()

In [4]:
# Query Data
def _query_db(query):
    print(f"Querying employees:\n===================\n")
    try:
        print(f"Query is: \n{query}\n")
        for emp in query.all():
            print(f"{emp.type}: {emp.name}")
            print(f"    Engineername: {emp.engineer_name}") if emp.type == 'engineer' else None
            print(f"    InfraEngname: {emp.engineer_name} {emp.db_password}") if emp.type == 'infraengineer' else None
            print(f"    Managername: {emp.manager_name}") if emp.type == 'manager' else None

    except Exception as e:
        print(f"Eror while querying {Book} {e}")
    print("\n\n")

query = session.query(Employee)
_query_db(query)

query = session.query(with_polymorphic(Employee, [Engineer]))
_query_db(query)

query = session.query(with_polymorphic(Employee, '*'))
_query_db(query)


Querying employees:

Query is: 
SELECT employee.id AS employee_id, employee.name AS employee_name, employee.type AS employee_type, engineer.id AS engineer_id, engineer.engineer_name AS engineer_engineer_name, manager.id AS manager_id, manager.manager_name AS manager_manager_name, infraengineer.id AS infraengineer_id, infraengineer.db_password AS infraengineer_db_password 
FROM employee LEFT OUTER JOIN engineer ON employee.id = engineer.id LEFT OUTER JOIN manager ON employee.id = manager.id LEFT OUTER JOIN infraengineer ON engineer.id = infraengineer.id

employee: Serena
engineer: Mariappan
    Engineername: Maari
engineer: Jarvis
    Engineername: Jar
infraengineer: Yong Wen
    InfraEngname: Yong password
infraengineer: Chris
    InfraEngname: Ch 12345
manager: Linus
    Managername: Li
manager: Feng
    Managername: Fe



Querying employees:

Query is: 
SELECT employee.id AS employee_id, employee.name AS employee_name, employee.type AS employee_type, engineer.id AS engineer_id, engin